In [ ]:
#Basic working model using collaborative filtering with watched, this is just removing the watched movies from list and rating genres from watched history

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.preprocessing import MinMaxScaler
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Sample user data
user_data = {
    'id': [1, 2, 3],
    'happy_movie': [['Drama', 'Comedy'], ['Romance', 'Family'], ['Adventure']],
    'sad_movie': [['Drama', 'Thriller'], ['Crime'], ['Horror', 'Science Fiction']],
    'neutral_movie': [['Comedy'], ['Action', 'Adventure'], ['Crime']],
    'fav_movie': ['The Godfather', "Schindler's List", 'The Shawshank Redemption']
}
user = pd.DataFrame(user_data)


# Load movie data from the specified file
file_path = '/content/drive/MyDrive/final project sample data/tmdb2024.csv'
movie1 = pd.read_csv(file_path, index_col=0)
#movie = minfo[['Movie_id', 'title', 'Genres', 'popularity', 'vote_average', 'vote_count']]



# Sample watched data
watched_data = {
    'userid': [1, 2, 3, 1, 2],
    'movieid': [157336, 155, 673629, 673629, 19995],
    'rating_given': [8.0, 5.5, 9.5, 7.0, 6.5],
    'watching_mood': ['Happy', 'Sad', 'Neutral', 'Happy', 'Neutral']
}
watched = pd.DataFrame(watched_data)

In [ ]:
movie1

,id,title,vote_average,vote_count,imdb_id,popularity,poster_path,Genres,Release_Year,Release_Era
0,27205,Inception,8.364,34495,tt1375666,83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,"['Action', 'Science Fiction', 'Adventure']",2010,mid
1,157336,Interstellar,8.417,32571,tt0816692,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,"['Adventure', 'Drama', 'Science Fiction']",2014,mid
2,155,The Dark Knight,8.512,30619,tt0468569,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,"['Drama', 'Action', 'Crime', 'Thriller']",2008,old
3,19995,Avatar,7.573,29815,tt0499549,79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",2009,old
4,24428,The Avengers,7.710,29166,tt0848228,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,"['Science Fiction', 'Action', 'Adventure']",2012,mid
...,...,...,...,...,...,...,...,...,...,...
1019124,673629,Black Wolf,0.000,0,tt10393330,0.616,/1E2wJPuaC8lN9EerSrN4Hwi2oa8.jpg,"['Action', 'Crime', 'Drama']",2022,latest
1019154,673384,Two Types: The Faces of Britain,0.000,0,tt7206158,0.600,/1sEz0xKALKiW3kmSw4tSYzVNpQM.jpg,['Documentary'],2017,mid
1019164,673405,Suddenly Homesick,0.000,0,tt11189058,0.600,/c8aA4gtVbKIqSZgEhAyaA5CRaEF.jpg,['Documentary'],2019,mid
1019166,673409,Lebbeus Woods + Steven Holl: The Practice of A...,0.000,0,tt2822964,0.600,/A2ITuES07G7HhaOzrtC2PTtI2Jp.jpg,['Documentary'],2012,mid


In [ ]:
movie1[movie1['id'] == 19404]

,id,title,vote_average,vote_count,imdb_id,popularity,poster_path,Genres,Release_Year,Release_Era


In [ ]:
pref_era='latest'

In [ ]:
movie = movie1[movie1['Release_Era'] == pref_era]


In [ ]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10274 entries, 282 to 1019124
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            10274 non-null  int64  
 1   title         10274 non-null  object 
 2   vote_average  10274 non-null  float64
 3   vote_count    10274 non-null  int64  
 4   imdb_id       10263 non-null  object 
 5   popularity    10274 non-null  float64
 6   poster_path   10267 non-null  object 
 7   Genres        10274 non-null  object 
 8   Release_Year  10274 non-null  int64  
 9   Release_Era   10274 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 882.9+ KB


In [ ]:
movie

,id,title,vote_average,vote_count,imdb_id,popularity,poster_path,Genres,Release_Year,Release_Era
282,76600,Avatar: The Way of Water,7.654,9830,tt1630029,241.285,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,"['Science Fiction', 'Adventure', 'Action']",2022,latest
366,414906,The Batman,7.705,8584,tt1877830,100.817,/74xTEgt7R36Fpooo50r9T25onhq.jpg,"['Crime', 'Mystery', 'Thriller']",2022,latest
403,453395,Doctor Strange in the Multiverse of Madness,7.355,7991,tt9419884,96.530,/9Gtg2DzBhmYamXBS1hKAhiwbBKS.jpg,"['Fantasy', 'Action', 'Adventure']",2022,latest
444,361743,Top Gun: Maverick,8.260,7546,tt1745960,126.291,/62HCnUTziyWcpDaBO2i1DX17ljH.jpg,"['Action', 'Drama']",2022,latest
527,502356,The Super Mario Bros. Movie,7.775,6743,tt6718170,410.411,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,"['Animation', 'Family', 'Adventure', 'Fantasy'...",2023,latest
...,...,...,...,...,...,...,...,...,...,...
1017425,676733,D-Day - Last Words,0.000,0,tt10399842,1.474,/48dSn6OoeMm5Cxyqxkb3YfFGVCf.jpg,"['War', 'Documentary']",2022,latest
1017827,674447,Justine,0.000,0,tt8816992,3.205,/2DxrrafoDNa68iK0dsss8pXQTDM.jpg,['Drama'],2022,latest
1018850,674254,Yatagan,0.000,0,tt5788218,0.626,/jv3DxblgXKFT40ZrifSALEw1bc.jpg,['Comedy'],2022,latest
1019060,673593,Mean Girls Musical,0.000,0,tt11762114,6.046,/1Lac9qrN6N4uO26T7nBgi8A7hVh.jpg,['Comedy'],2024,latest


In [ ]:
# Convert the string representation of list to actual list then to list of dictionaries
movie['Genres'] = movie['Genres'].apply(lambda x: [{'genre': genre} for genre in ast.literal_eval(x)])

# Extract the names of the genres from the list of dictionaries
movie['Genres'] = movie['Genres'].apply(lambda x: [i['genre'] for i in x])

<ipython-input-152-7ee3f36e7acf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['Genres'] = movie['Genres'].apply(lambda x: [{'genre': genre} for genre in ast.literal_eval(x)])
<ipython-input-152-7ee3f36e7acf>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['Genres'] = movie['Genres'].apply(lambda x: [i['genre'] for i in x])


In [ ]:
movie.head(2)

,id,title,vote_average,vote_count,imdb_id,popularity,poster_path,Genres,Release_Year,Release_Era
282,76600,Avatar: The Way of Water,7.654,9830,tt1630029,241.285,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,"[Science Fiction, Adventure, Action]",2022,latest
366,414906,The Batman,7.705,8584,tt1877830,100.817,/74xTEgt7R36Fpooo50r9T25onhq.jpg,"[Crime, Mystery, Thriller]",2022,latest


In [ ]:
movie_genres = movie.copy()

In [ ]:
movie_genres.rename(columns={'id': 'Movie_id'}, inplace=True)

In [ ]:
movie_genres

,Movie_id,title,vote_average,vote_count,imdb_id,popularity,poster_path,Genres,Release_Year,Release_Era
282,76600,Avatar: The Way of Water,7.654,9830,tt1630029,241.285,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,"[Science Fiction, Adventure, Action]",2022,latest
366,414906,The Batman,7.705,8584,tt1877830,100.817,/74xTEgt7R36Fpooo50r9T25onhq.jpg,"[Crime, Mystery, Thriller]",2022,latest
403,453395,Doctor Strange in the Multiverse of Madness,7.355,7991,tt9419884,96.530,/9Gtg2DzBhmYamXBS1hKAhiwbBKS.jpg,"[Fantasy, Action, Adventure]",2022,latest
444,361743,Top Gun: Maverick,8.260,7546,tt1745960,126.291,/62HCnUTziyWcpDaBO2i1DX17ljH.jpg,"[Action, Drama]",2022,latest
527,502356,The Super Mario Bros. Movie,7.775,6743,tt6718170,410.411,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,"[Animation, Family, Adventure, Fantasy, Comedy]",2023,latest
...,...,...,...,...,...,...,...,...,...,...
1017425,676733,D-Day - Last Words,0.000,0,tt10399842,1.474,/48dSn6OoeMm5Cxyqxkb3YfFGVCf.jpg,"[War, Documentary]",2022,latest
1017827,674447,Justine,0.000,0,tt8816992,3.205,/2DxrrafoDNa68iK0dsss8pXQTDM.jpg,[Drama],2022,latest
1018850,674254,Yatagan,0.000,0,tt5788218,0.626,/jv3DxblgXKFT40ZrifSALEw1bc.jpg,[Comedy],2022,latest
1019060,673593,Mean Girls Musical,0.000,0,tt11762114,6.046,/1Lac9qrN6N4uO26T7nBgi8A7hVh.jpg,[Comedy],2024,latest


In [ ]:
# prompt: Using dataframe movie: i want Genres to be reformatted with only list of name of genres, right now it has id and name and addtional text

# import ast

# # Create a copy of the dataframe
# movie_genres = movie.copy()

# # Convert the Genres column from string to list of dictionaries
# movie_genres['Genres'] = movie_genres['Genres'].apply(ast.literal_eval)

# # Extract the names of the genres from the list of dictionaries
# movie_genres['Genres'] = movie_genres['Genres'].apply(lambda x: [i['name'] for i in x])

# # Display the first 10 rows of the dataframe
# movie_genres.head(10)


In [ ]:
# Assuming user, watched, and movie DataFrames are already defined as per your description.

# Merge the watched movies data with the movie details
watched_movies = pd.merge(watched, movie_genres, left_on='movieid', right_on='Movie_id')



In [ ]:
watched_movies

,userid,movieid,rating_given,watching_mood,Movie_id,title,vote_average,vote_count,imdb_id,popularity,poster_path,Genres,Release_Year,Release_Era
0,3,673629,9.5,Neutral,673629,Black Wolf,0.0,0,tt10393330,0.616,/1E2wJPuaC8lN9EerSrN4Hwi2oa8.jpg,"[Action, Crime, Drama]",2022,latest
1,1,673629,7.0,Happy,673629,Black Wolf,0.0,0,tt10393330,0.616,/1E2wJPuaC8lN9EerSrN4Hwi2oa8.jpg,"[Action, Crime, Drama]",2022,latest


In [ ]:

from itertools import chain

# Assuming 'user' is your DataFrame containing user preferences.
# Sample data structure for 'user' DataFrame is provided in your initial description.

# Function to expand and tag mood to genres
def expand_user_genres(user_df, mood_column):
    # Explode the DataFrame by the specified mood to handle multiple genres
    expanded_df = user_df[['id', mood_column]].explode(mood_column)
    # Assign the current mood as a new column
    expanded_df['mood'] = mood_column.split('_')[0].capitalize()
    # Rename the genre column to a generic 'genre' column
    expanded_df = expanded_df.rename(columns={mood_column: 'genre'})
    return expanded_df

# Process each mood
happy_genres = expand_user_genres(user, 'happy_movie')
sad_genres = expand_user_genres(user, 'sad_movie')
neutral_genres = expand_user_genres(user, 'neutral_movie')

# Concatenate all moods together
all_genres = pd.concat([happy_genres, sad_genres, neutral_genres])

# Optional: Reset index if you want a clean DataFrame index
all_genres.reset_index(drop=True, inplace=True)

all_genres.head()


,id,genre,mood
0,1,Drama,Happy
1,1,Comedy,Happy
2,2,Romance,Happy
3,2,Family,Happy
4,3,Adventure,Happy


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Assuming movie_genres DataFrame has a 'Genres' column with lists of genres
mlb = MultiLabelBinarizer()
movie_genres_encoded = mlb.fit_transform(movie_genres['Genres'])
movie_genres_df = pd.DataFrame(movie_genres_encoded, columns=mlb.classes_, index=movie_genres['Movie_id'])



In [ ]:
movie_genres_df

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
Movie_id,,,,,,,,,,,,,,,,,,,
76600,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
414906,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0
453395,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
361743,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
502356,0,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676733,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
674447,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
674254,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# def recommend_movies_with_ratings_and_mood(user_id, current_mood, top_n=5):
#     # Get genres the user prefers in the current mood
#     mood_genres = all_genres[(all_genres['id'] == user_id) & (all_genres['mood'] == current_mood)]
#     user_pref_genres = mood_genres['genre'].unique()

#     # Get movies watched and rated by the user in the specified mood
#     mood_watched_movies = watched_movies[(watched_movies['userid'] == user_id) & (watched_movies['watching_mood'].str.capitalize() == current_mood)]

#     # Calculate average ratings for genres in this mood

#     mood_watched_movies_details = mood_watched_movies.merge(movie_genres, left_on='movieid', right_index=True)

#     # Adjusted section to use 'Genres_x'
#     genre_ratings = {}
#     for genre in user_pref_genres:
#         # Here we filter using 'Genres_x'
#         genre_movies = mood_watched_movies_details[mood_watched_movies_details['Genres_x'].apply(lambda x: genre in x)]
#         avg_rating = genre_movies['rating_given'].mean()
#         genre_ratings[genre] = avg_rating if not np.isnan(avg_rating) else 0


#     # Rank genres by average rating
#     ranked_genres = sorted(genre_ratings, key=genre_ratings.get, reverse=True)

#     # Encode user preferences weighted by average genre rating
#     user_pref_encoded = mlb.transform([ranked_genres])

#     # Calculate similarity between weighted user preferences and each movie's genres
#     similarity = cosine_similarity(user_pref_encoded, movie_genres_encoded)

#     # Exclude movies already watched
#     watched_movie_ids = watched_movies[watched_movies['userid'] == user_id]['movieid'].unique()
#     potential_movies = movie_genres[~movie_genres.index.isin(watched_movie_ids)]

#     # Get top N recommended movie IDs based on similarity scores, excluding watched movies
#     recommended_movie_ids = similarity.argsort()[0][-top_n:][::-1]

#     # Ensure correct indexing and data type consistency for recommended_movie_ids
#     recommended_movie_ids = [potential_movies.index[i] for i in recommended_movie_ids if i < len(potential_movies)]

#     # Map movie IDs back to movie titles and genres
#     recommended_movies_info = movie_genres.loc[recommended_movie_ids][['title', 'Genres']]

#     # Create a more readable output format
#     recommended_movies_output = recommended_movies_info.apply(lambda x: f"{x['title']} (Genres: {', '.join(x['Genres'])})", axis=1)

#     return recommended_movies_output.tolist()

# # Example usage
# recommended_movies = recommend_movies_with_ratings_and_mood(1, 'Happy', 5)
# for movie in recommended_movies:
#     print(movie)


In [ ]:
movie_genres.set_index('Movie_id', inplace=True)

In [ ]:
def recommend_movies_with_weighted_genres(user_id, current_mood, top_n=5):
    mood_genres = all_genres[(all_genres['id'] == user_id) & (all_genres['mood'] == current_mood)]
    user_pref_genres = mood_genres['genre'].unique()

    # Check for mood-specific watched movies
    mood_watched_movies = watched_movies[(watched_movies['userid'] == user_id) & (watched_movies['watching_mood'].str.capitalize() == current_mood)]

    genre_ratings = {}
    if not mood_watched_movies.empty:
        # Calculate average ratings for genres from mood-specific watched movies
        mood_watched_movies_details = mood_watched_movies.merge(movie_genres, left_on='movieid', right_index=True)
        for genre in user_pref_genres:
            genre_movies = mood_watched_movies_details[mood_watched_movies_details['Genres_x'].apply(lambda x: genre in x)]
            avg_rating = genre_movies['rating_given'].mean()
            genre_ratings[genre] = avg_rating if not np.isnan(avg_rating) else 0
        # Normalize ratings to use as weights
        max_rating = max(genre_ratings.values(), default=1)
        genre_weights = {genre: rating / max_rating for genre, rating in genre_ratings.items()}
    else:
        genre_weights = {genre: 1 for genre in user_pref_genres}  # Default weight if no ratings

    # Encode movie genres and user preferences
    mlb = MultiLabelBinarizer()
    movie_genres_encoded = mlb.fit_transform(movie_genres['Genres'])
    genre_prefs_encoded = mlb.transform([user_pref_genres])

    # Calculate similarity with weighted genres
    similarity_scores = cosine_similarity(genre_prefs_encoded, movie_genres_encoded)[0]
    weighted_similarity_scores = np.zeros_like(similarity_scores)

    for i, genre in enumerate(mlb.classes_):
        if genre in genre_weights:
            weighted_similarity_scores += similarity_scores * movie_genres_encoded[:, i] * genre_weights[genre]

    # Normalize popularity, vote_average and scale them

    scaler = MinMaxScaler()
    normalized_popularity = scaler.fit_transform(movie_genres[['popularity']])
    normalized_vote_average = scaler.fit_transform(movie_genres[['vote_average']])


    # Calculate final score combining popularity, vote average and weighted similarity scores
    final_scores = weighted_similarity_scores + normalized_popularity.flatten() + normalized_vote_average.flatten()

    # Exclude watched movies
    watched_movie_ids = watched_movies[watched_movies['userid'] == user_id]['movieid'].unique()
    unwatched_indices = [i for i, movie_id in enumerate(movie_genres.index) if movie_id not in watched_movie_ids]

    # Recommend based on weighted similarity scores
    unwatched_scores = final_scores[unwatched_indices]
    top_indices = np.argsort(unwatched_scores)[-top_n:][::-1]

    recommended_movie_ids = [movie_genres.index[unwatched_indices[i]] for i in top_indices]
    recommended_movies_info = movie_genres.loc[recommended_movie_ids]
    #recommended_movies_output = recommended_movies_info.apply(lambda x: f"{x['title']} (Genres: {', '.join(x['Genres'])}), Popularity: {x['popularity']:.2f}, Average Vote: {x['vote_average']:.1f}", axis=1).tolist()
    recommended_movies_output = recommended_movies_info['title'].tolist()

    return recommended_movies_output

# Example usage
recommended_movies = recommend_movies_with_weighted_genres(1, 'Happy', 100)
for movie in recommended_movies:
    print(movie)



Gran Turismo
Before I Change My Mind
Little Jar
The Duel
The Interview
Written By
Moth!
Clam Shack Blues
Mechanical Bulls
First Time
What's My Name Again?
Mustache
Florida Man
American Dreams Redux
Chicken
Sharpie
The Quarter Life Dolphins
Cold Little Bird
Unpacking
Shellfish
Half
The Hypnosis
You Can't Fight in Here
Tundra Town: The Legend Goes On
Hungarian Scouts
Changing Signs
The Bear
Attendant - Shell of a Life
The Debacle of Mr. Eastward
My Apocalyptic Thanksgiving
Moon Manor
Flamin' Hot
Amerikatsi
Sell/Buy/Date
Desi Standard Time Travel
The Jerry Condition
Relative
A Man Called Otto
S.Q.A.G. (Short Quiet Asian Girl)
The Curse of Professor Zardonicus
Meeting Mr Samuel
Hustle
Wildflower
Babylon
Dog
Lebensdorf
Are You There God? It's Me, Margaret.
Therapy Dogs
Jerry & Marge Go Large
Three Months
Un parcero en Nueva York
The Blind
Triangle of Sadness
Mrs. Harris Goes to Paris
Queen of Glory
Simón
The Man from Rome
Confessions of a Superhero
Julia(s)
Blue Beetle
Dream
The Salamander


In [ ]:
# Get recommended movie titles
recommended_movies = recommend_movies_with_weighted_genres(1, 'Happy', 100)

# Initialize an empty list to store the DataFrame rows
recommended_movies_list = []

# Collect the entire row for each recommended movie from the movie_genres DataFrame
for title in recommended_movies:
    movie_row = movie_genres[movie_genres['title'] == title]
    recommended_movies_list.append(movie_row)

# Concatenate all the collected rows into a single DataFrame
recommended_movies_df = pd.concat(recommended_movies_list)

recommended_movies_df = recommended_movies_df.sort_values(by='popularity', ascending=False)


# Print the resulting DataFrame
print(recommended_movies_df)


                                title  vote_average  vote_count     imdb_id  \
Movie_id                                                                      
565770                    Blue Beetle         7.139        1023   tt9362930   
980489                   Gran Turismo         8.068         702   tt4495098   
937278              A Man Called Otto         7.814        1857   tt7405458   
626332                    Flamin' Hot         8.186         400   tt8105234   
615777                        Babylon         7.443        2244  tt10640346   
...                               ...           ...         ...         ...   
1049996   My Apocalyptic Thanksgiving         9.000           1   tt6205888   
1148518   The Debacle of Mr. Eastward         9.000           1  tt28137674   
1133277        I Don't Like Gardening        10.000           1  tt27687952   
1209991   Attendant - Shell of a Life        10.000           1  tt28837936   
1207091             Meeting Mr Samuel         8.000 

In [ ]:
# Example: Recommend movies for user 1 in a "Happy" mood, including genres
recommended_movies = recommend_movies_with_weighted_genres(1, 'Happy', 10)
for movie in recommended_movies:
    print(movie)

Gran Turismo
Before I Change My Mind
Little Jar
The Duel
The Interview
Written By
Moth!
Clam Shack Blues
Mechanical Bulls
First Time


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(1, 'Sad', 10)
for movie in recommended_movies:
    print(movie)

The Stepmother 3
Sweet Sting of the Black Widow
First Comes Love, Then Comes Murder
Happy Suicide
Birdeater
CACOPHONY
SpeculatorS
Connie Lynn
The Cost
Woodsman


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(1, 'Neutral', 10)
for movie in recommended_movies:
    print(movie)

Quiz Lady
Office Games
Jon Stewart: The Kennedy Center Mark Twain Prize
Phil Gunderson Teaches Screenwriting
Babe Beach
Devolver Marketing Countdown to Marketing
Canned
What is Art
Death Toilet 5: Invasion of the Potty Snatchers
Meatmen


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(2, 'Happy', 200)
for movie in recommended_movies:
    print(movie)


Holly and the Hot Chocolate
Learning to Love
Horse Camp: A Love Tail
Vicious Affair
Elemental
The Presence of Love
Come Fly with Me
Falling for Christmas
The Little Mermaid
Lease on Love
The Invisible Thread
Erin's Guide to Kissing Girls
Meet Me in New York
The Attraction Test
The Story of Love
Love, Fashion, Repeat
Christmas Plus One
Cider and Sunsets
Planes, Trains, and Christmas Trees
All I Want For Christmas
Styled with Love
Grumpy Old Santa
Love, Game, Match
Scented with Love
The Engagement Back-Up
My Heavenly City
Love Amongst the Stars
A Slice of Chicago Romance
Sweet as Pie
Planning On Forever
Beauty and the Beast: A 30th Celebration
Love & Where to Find It
Love for Starters
A Royal Christmas Match
Love and the Radio Star
Zoe's Having a Baby
Love Under the Lemon Tree
Loving Christmas
The Love Subscription
Planning for Joy
Hanukkah on Rye
The Song to My Heart
A Priceless Love
Blue Beetle
I Love Lizzy
Lesbian Sex 25
Taboo Love: Brothers Loving Stepsisters
Lesbian Seductions: Olde

In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(2, 'Sad', 10)
for movie in recommended_movies:
    print(movie)

Shadows
The Outfit
Blue Beetle
Operation Mayfair
Cascade
Lottery
The Real Story of January 6
Unlicensed
Jade. A White Coral Spinoff
The Sexual Torture Chamber of Doctor Josefel


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(2, 'Neutral', 10)
for movie in recommended_movies:
    print(movie)

Blue Beetle
Gran Turismo
Tozoom
Indiana Jones and the Dial of Destiny
What We Become
Uncharted
Spider-Man: Across the Spider-Verse
LEGO Monkie Kid: The Emperor's Wrath
LEGO Monkie Kid: Embrace Your Destiny
MIRACULUM


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(3, 'Happy', 10)
for movie in recommended_movies:
    print(movie)

Blue Beetle
Gran Turismo
Always Now
Tozoom
Barbie
Carl's Date
The Highwayman
Strays
Spider-Man: Across the Spider-Verse
THE QUEST: Nepal


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(3, 'Sad', 10)
for movie in recommended_movies:
    print(movie)

Serpentine
Meg 2: The Trench
Piranha Women
Subject
Exo Sapien
CD-Trip
Six
M3GAN
Wakener
Raven


In [ ]:
recommended_movies = recommend_movies_with_weighted_genres(3, 'Neutral', 10)
for movie in recommended_movies:
    print(movie)

Shadows
The Outfit
Blue Beetle
Operation Mayfair
Cascade
Lottery
The Real Story of January 6
Unlicensed
Jade. A White Coral Spinoff
The Sexual Torture Chamber of Doctor Josefel


In [ ]:
user

,id,happy_movie,sad_movie,neutral_movie,fav_movie
0,1,"[Drama, Comedy]","[Drama, Thriller]",[Comedy],The Godfather
1,2,"[Romance, Family]",[Crime],"[Action, Adventure]",Schindler's List
2,3,[Adventure],"[Horror, Science Fiction]",[Crime],The Shawshank Redemption


Relevant code ends here